# Lab 6: Logistic Regression & Auto-differentiation

**Sources**: Part A is based on the [one neuron](https://github.com/cbernet/maldives/tree/master/one_neuron) notebooks by  [Colin Bernet](https://github.com/cbernet). Part B is based on the PyTorch introduction lab by Juan-José Giraldo and Mauricio A Álvarez.

**Note**: Try to answer the questions when you first see them rather than coming back after going through the rest.

# <a id='partA'></a>A: Logistic Regression

In this first part we will look at programming some aspects of logistic regression but also use the Scikit-Learn implementation of it on a simple test dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## A1. The sigmoid function 

The **sigmoid** or **logistic function** is essential in binary classification problems. It is expressed as
$$\sigma(z) = \frac{1}{1+e^{-z}}$$
and here is what it looks like in 1D:

In [ ]:
# define parameters
# the bias: 
b = 0
# the weight: 
w = 1

def sigmoid(x1):
    # z is a linear function of x1
    z = w*x1 + b
    return 1 / (1+np.exp(-z))

# create an array of evenly spaced values
linx = np.linspace(-10,10,51)
plt.plot(linx, sigmoid(linx))
b=5
plt.plot(linx, sigmoid(linx))
w=5
plt.plot(linx, sigmoid(linx))

---
### Question 1

What are the parameters (w and b) for each of the three curves orange (left), green (middle) and blue (right)?

**Answer**: 

---

Let's look at this function in more details:

* when $z$ goes to infinity, $e^{-z}$ goes to zero, and $\sigma (z)$ goes to one.
* when $z$ goes to minus infinity, $e^{-z}$ goes to infinity, and $\sigma (z)$ goes to zero.
* $\sigma(0) = 0.5$, since $e^0=1$.

It is important to note that the sigmoid is bound between 0 and 1, like a probability. And actually, in binary classification problems, the probability for an example to belong to a given category is produced by a sigmoid function. To classify our examples, we can simply use the output of the sigmoid: A given unknown example with value $x$ will be classified to category 1 if $\sigma(z) > 0.5$, and to category 0 otherwise. 

**Excercise**: Now you can go back to the cell above, and play a bit with the `b` and `w` parameters, redoing the plot everytime you change one of these parameters. 

* $b$ is the **bias**. Changing the bias simply moves the sigmoid along the horizontal axis. For example, if you choose $b=1$ and $w=1$, then $z = wx + b = 0$ at $x=-1$, and that's where the sigmoid will be equal to 0.5
* $w$ is the **weight** of variable $x$. If you increase it, the sigmoid evolves faster as a function of $x$ and gets sharper.

## A2. Logistic regression as the simplest neural network

We will build the simplest neural network to classify our examples:

* Each example has one variable, so we need 1 input node on the input layer
* We're not going to use any hidden layer, as that would complicate the network 
* We have two categories, so the output of the network should be a single value between 0 and 1, which is the estimated probability $p$ for an example to belong to category 1. Then, the probability to belong to category 0 is simply $1-p$. Therefore, we should have a single output neuron, the only neuron in the network.

The sigmoid function can be used in the output neuron. Indeed, it spits out a value between 0 and 1, and can be used as a classification probability as we have seen in the previous section.

We can represent our network in the following way:

![Neural network with 1 neuron](https://github.com/cbernet/maldives/raw/master/images/one_neuron.png)

In the output neuron: 

* the first box performs a change of variable and computes the **weighted input** $z$ of the neuron
* the second box applies the **activation function** to the weighted input. Here, we choose the sigmoid $\sigma (z) = 1/(1+e^{-z})$ as an activation function

This simple network has only 2 tunable parameters, the weight $w$ and the bias $b$, both used in the first box. We see in particular that when the bias is very large, the neuron will **always be activated**, whatever the input. On the contrary, for very negative biases, the neuron is **dead**. 

We can write the output simply as a function of $x$, 

$$f(x) = \sigma(z) = \sigma(wx+b)$$
This is exactly the **logistic regression** classifier.

---
### Question 2

How can we rewrite the logistic regression classifier above using a **single vectorial parameter** (i.e. one vector containing all parameters)?

**Answer**: 

---

## A3. Classifying 2D dataset with logistic regression

Let's create a sample of examples with two values x1 and x2, with two categories. 
For category 0, the underlying probability distribution is a 2D Gaussian centered on (0,0), with width = 1 along both directions. For category 1, the Gaussian is centered on (2,2). We assign label 0 to category 0, and label 1 to category 1.

### Dataset creation

Let's create a sample of examples with two values x1 and x2, with two categories. 
For category 0, the underlying probability distribution is a 2D Gaussian centered on (0,0), with width = 1 along both directions. For category 1, the Gaussian is centered on (2,2). We assign label 0 to category 0, and label 1 to category 1. Check out the [documentation for Gaussian data generation](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.multivariate_normal.html)

In [ ]:
normal = np.random.multivariate_normal
# Number of samples 
nSamples = 500
# (unit) variance:
s2 = 1
# below, we provide the coordinates of the mean as 
# a first argument, and then the covariance matrix
# we generate nexamples examples for each category
sgx0 = normal([0.,0.], [[s2, 0.], [0.,s2]], nSamples)
sgx1 = normal([2.,2.], [[s2, 0.], [0.,s2]], nSamples)
# setting the labels for each category
sgy0 = np.zeros((nSamples,))
sgy1 = np.ones((nSamples,))

Here is a scatter plot for the examples in the two categories

In [ ]:
plt.scatter(sgx0[:,0], sgx0[:,1], alpha=0.5)
plt.scatter(sgx1[:,0], sgx1[:,1], alpha=0.5)
plt.xlabel('x1')
plt.ylabel('x2')

Our goal is to train a logistic regression to classify (x1,x2) points in one of the two categories depending on the values of x1 and x2. We form the dataset by concatenating the arrays of points, and also the arrays of labels for later use:

In [ ]:
sgx = np.concatenate((sgx0, sgx1))
sgy = np.concatenate((sgy0, sgy1))

print(sgx.shape, sgy.shape)

### 2D sigmoid

In 2D, the expression of the sigmoid remains the same, but $z$ is now a function of the two variables $x_1$ and $x_2$, 

$$z=w_1 x_1 + w_2 x_2 + b$$

And here is the code for the **2D sigmoid** and the defined function is called **sigmoid_2d**:

In [ ]:
# define parameters
# bias: 
b = 0
# x1 weight: 
w1 = 1
# x2 weight:
w2 = 2

def sigmoid_2d(x1, x2):
    # z is a linear function of x1 and x2
    z = w1*x1 + w2*x2 + b
    return 1 / (1+np.exp(-z))

To see what this function looks like, we can make a 2D plot, with x1 on the horizontal axis, x2 on the vertical axis, and the value of the sigmoid represented as a color for each (x1, x2) coordinate. To do that, we will create an array of evenly spaced values along x1, and another array along x2. Taken together, these arrays will allow us to map the (x1,x2) plane. 

In [ ]:
xmin, xmax, npoints = (-6,6,51)
linx1 = np.linspace(xmin,xmax,npoints)
# no need for a new array, we just reuse the one we have with another name: 
linx2 = linx1

Then, we create a **meshgrid** from these arrays: 

In [ ]:
gridx1, gridx2 = np.meshgrid(np.linspace(xmin,xmax,npoints), np.linspace(xmin,xmax,npoints))
print(gridx1.shape, gridx2.shape)
print('gridx1:')
print(gridx1) 
print('gridx2')
print(gridx2)

if you take the first line in both arrays, and scan the values on this line, you get: `(-6,-6), (-5.76, -6), (-5.52, -6)`... So we are scanning the x1 coordinates sequentially at the bottom of the plot. If you take the second line, you get: `(-6, -5.76), (-5.76, -5.76), (-5.52, -5.76)` ... : we are scanning the second line at the bottom of the plot, after moving up in x2 from one step. 

Scanning the full grid, you would scan the whole plot sequentially. 

Now we need to compute the value of the sigmoid for each pair (x1,x2) in the grid using the **sigmoid_2d** function defined above (cell 6). That's very easy to do with the output of `meshgrid`: 

In [ ]:
z = sigmoid_2d(gridx1, gridx2)
z.shape

This calls the `sigmoid_2d` function to each pair `(x1,y2)` taken from the `gridx1` and `gridx2` arrays so that we can plot our sigmoid in 2D: 

In [ ]:
plt.contourf(gridx1, gridx2, z)
plt.xlabel('x1')
plt.ylabel('x2')
plt.colorbar()

The 2D sigmoid has the same kind of rising edge as the 1D sigmoid, but in 2D. 
With the parameters defined above: 

* The **weight** of $x_2$ is twice larger than the weight of $x_1$, so the sigmoid evolves twice faster as a function of $x_2$. 
* The separation boundary, which occurs for $z=0$, is a straight line with equation $w_1 x_1 + w_2 x_2 + b = 0$. Or equivalently: 

$$x_2 = -\frac{w_1}{w_2} x_1 - \frac{b}{w_2} = -0.5 x_1$$

---
### Question 3

If you set one of the weights to zero, what will happen? Also, verify on the plot above that the equation above is indeed the one describing the separation boundary. 

**Answer**: 

---

Now you can test by editing the function `sigmoid_2d`, before re-executing the above cells. 

Note that if you prefer, you can plot the sigmoid in 3D like this:  

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_wireframe(gridx1,gridx2,z)

**Exercise**: change the parameters to observe how the 2D sigmoid changes.

### Logistic regression on the 2D data

Let's now train a logistic regression to separate the two classes of examples. The goal of the training will be to use the existing examples to find the optimal values for the parameters $w_1, w_2, b$. 

We take the logistic regression algorithm from scikit-learn. 
Here, the logistic regression is used with the `lbfgs` solver. LBFGS is the minimization method used to find the best parameters. It is similar to [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization). Since there is randomness, setting a seed is a good practice for **reproducibility**.

In [ ]:
from sklearn.linear_model import LogisticRegression
np.random.seed(2020) #set a seed for reproducibility
clf = LogisticRegression(solver='lbfgs')  #clf: classifier
clf.fit(sgx, sgy)

In [ ]:
sgx

Note from the above that the default setting for logistic regression in scikit-learn uses L2 regularisation (penalty). 

---
### Question 4

What is the objective of L2 regularisation (penalty)? *Hint*: this is not covered in lecture and you need to do some study (search).

**Answer**: 

---

Check out the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) to learn other options for penalty (regularisation) and other settings. In the simplest form, logistic regression does not have any hyperparameters but in practice, regularisation is often used, e.g. to reduce [overfitting](https://en.wikipedia.org/wiki/Overfitting).

The logistic regression has been fitted (trained) to the data. Now, we can use it to predict the probability for any given (x1,x2) point to belong to category 1.

We would like to plot this probability in 2D as a function of x1 and x2. To do that, we need to use the `clf.predict_proba` method which takes a 2D array of shape `(n_points, 2)`. The first dimension indexes the points, and the second one contains the values of x1 and x2. Again, we use our grid to map the (x1,x2) plane. But the gridx1 and gridx2 arrays defined above contain disconnected values of x1 and x2: 

In [ ]:
print(gridx1.shape, gridx2.shape)

What we want is a 2D array of shape `(n_points, 2)`, not two 2D arrays of shape (51, 51)... 
So we need to **reshape** these arrays. First, we will **flatten** the gridx1 and gridx2 arrays so that all their values appear sequentially in a 1D array. Here is a small example to show how flatten works: 

In [ ]:
a = np.array([[0, 1], [2, 3]])
print(a) 
print('flat array:', a.flatten())

Then, we will stitch the two 1D arrays together in two columns with np.c_ like this: 

In [ ]:
b = np.array([[4, 5], [6, 7]])
print(a.flatten())
print(b.flatten())
c = np.c_[a.flatten(), b.flatten()]
print(c)
print(c.shape)

This array has exactly the shape expected by `clf.predict_proba`: a list of samples with two values. So let's do the same with our meshgrid, and let's compute the probabilities for all (x1,x2) pairs in the grid:

In [ ]:
grid = np.c_[gridx1.flatten(), gridx2.flatten()]
prob = clf.predict_proba(grid)
prob.shape

Now, prob does not have the right shape to be plotted. Below, we will use a gridx1 and a gridx2 array with shapes (51,51). The shape of the prob array must also be (51,51), as the plotting method will simply map each (x1,x2) pair to a probability. So we need to **reshape** our probability array to shape (51,51). Reshaping works like this:

In [ ]:
d = np.array([0,1,2,3])
print(d)
print('reshaped to (2,2):')
print(d.reshape(2,2))

Finally (!) we can do our plot:

In [ ]:
# note that prob[:,1] returns, for all exemples, the probability p to belong to category 1. 
# prob[:,0] would return the probability to belong to category 0 (which is 1-p)
plt.contourf(gridx1,gridx2,prob[:,1].reshape(npoints,npoints))
plt.colorbar()
plt.scatter(sgx0[:,0], sgx0[:,1], alpha=0.5)
plt.scatter(sgx1[:,0], sgx1[:,1], alpha=0.5)
plt.xlabel('x1')
plt.ylabel('x2')

We see that the logistic regression is able to separate these two classes well and the decision boundary is **linear**.

# B: Auto-differentiation with PyTorch

7th November 2022, adapted from book by Juan-José Giraldo, Mauricio A Álvarez and Haiping Lu.

In this Notebook, we look at the torch library in Python that allows automatic differentiation. PyTorch will be used to implement different neural network models later on.

**Suggested reading**: 
* What is PyTorch from [PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py)

**Assumptions** : basic python programming and [Anaconda](https://anaconda.org/) installed.


## PyTorch Installation and Basics

If you are running on Google Colab then PyTorch will already be installed but you may need to install it on your own machines. There are different options on the PyTorch homepage for downloading it including conda and pip. Here we will describe how to install it using conda.



### Install-1: direct installation (e.g., on your own machine with full installation right)

**Install [PyTorch](https://github.com/pytorch/pytorch) via [Anaconda](https://anaconda.org/)**
`conda install -c pytorch pytorch`

 When you are asked whether to proceed, say `y`

**Install [torchvision](https://github.com/pytorch/vision)**
`conda install -c pytorch torchvision`

 When you are asked whether to proceed, say `y`

### Install-2: Set up Anaconda Python environment (e.g., on a university desktop)

On a university desktop, you may not have permission to install new packages on the main environment of Anaconda. Please follow the instructions below to set up a new environment. This is also recommended if you have different python projects running that may require different environments.

Open a command line terminal.

**Create a new conda environment with Python 3.8**<br>
`conda create -n mlai python=3.8 anaconda`

**Activate the conda environment `mlai`** (see [conda documentation](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-environments))<br>

For conda 4.6 and later versions: `conda activate mlai`

For conda versions prior to 4.6<br>
`activate mlai` (Windows)<br>
`source activate mlai` (Mac/Linux)<br><br>
You will see `(mlai)` on the left indciating your environment

**Install Pytorch and Torchvision** (non-CUDA/GPU version for simplicity)<br>
`conda install pytorch torchvision cpuonly -c pytorch`<br>
If you have GPU, install the GPU version with command at [here](https://pytorch.org/)

**Start Jupyter notebook server**: `jupyter notebook`

## B1: Tensors
A tensor generalises the concept of vectors and matrices to an arbitrary number of dimensions. Another name for the same concept is multidimensional arrays. The dimensionality of a tensor is the number of indexes used to refer to scalar values within the tensor. The cell below shows an example initialising a Tensor uniformly for 1D, 2D and 3D:

In [ ]:
# We first import the torch library that comes with the Anaconda distribution
import torch 
# Tensor 1D presents 1 index
y = torch.rand([2])
print('Tensor 1D presents one index','with shape', y.shape,':\n',y) #get specific size with .shape
# Tensor 2D presents 2 indexes
y = torch.rand([2,3])
print('\nTensor 2D presents two indexes','with shape',y.shape,':\n',y)
#Tensor 3D presents 3 indexes
y = torch.rand([5,2,3])
print('\nTensor 3D presents three indexes','with shape',y.shape,':\n',y)

In [ ]:
# Create a tensor with specific values
x = torch.tensor([4.0,5.0],dtype=torch.float32)
y = torch.tensor([2.0,3.0],dtype=torch.float32)

# Tensor multiplication (point-wise multiplication)
print(x*y)

### Initialise a tensor with torch.zeros or torch.ones 

In [ ]:
x_zeros = torch.zeros([3,4])
print('x_zeros:',x_zeros,'with shape',x_zeros.shape,'\n')
x_ones = torch.ones([2,6])
print('x_ones:',x_ones,'with shape',x_ones.shape)

### Reshape a tensor using .view

In [ ]:
y = torch.ones([3,2])
y_reshaped = y.view(6,1)  # in contrast to the common numpy library, we use .view instead of .reshape
print(y_reshaped)

## B2: Numpy interoperability 

PyTorch tensors can be converted efficiently to NumPy arrays and vice versa. By doing so, you can leverage the huge swath of functionality in the wider Python ecosystem that has built up around the NumPy array type.

In [ ]:
# Tensor_torch to tensor Numpy
Tensor_torch = torch.ones(3,4)
Tensor_numpy = Tensor_torch.numpy() #Returns a NumPy multidim. array of the right size, shape and numerical type.
print('Array in numpy form with shape', Tensor_numpy.shape,':\n',Tensor_numpy)

# Tensor Numpy to Tensor_torch
import numpy as np
Tensor_np = np.random.randn(5,8)    
Tensor_numpy_to_torch = torch.from_numpy(Tensor_np)
print('\nArray from Numpy to Torch with shape', Tensor_numpy_to_torch.shape,':\n',Tensor_numpy_to_torch)

### Loading a .csv dataset

We can take advantage of the interoperability between Numpy and PyTorch by loading a .csv data as a numpy array and transforming it to a Torch Tensor using `torch.from_numpy(dataset_np)`. 

In [ ]:
# This cell is simply to download the winequality-red.csv dataset from its root url
import urllib.request
urllib.request.urlretrieve('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', './winequality-red.csv')

In [ ]:
import numpy as np
#In the line below we avoid the first row (skiprows=1) of .csv file that contains names
#the delimeter of data for this dataset is ";"
wine_np = np.loadtxt("./winequality-red.csv",dtype=np.float32,delimiter=";",skiprows=1)
wine_torch = torch.from_numpy(wine_np)  #We take advantage of the interoperability with numpy
wine_torch

## B3: Automatic Differentiation





### Computational Graph


A computation graph defines/visualises a sequence of operations to go from input to model output. 

Consider a linear regression model $\hat y = Wx + b$, where $x$ is our input, $W$ is a weight matrix, $b$ is a bias, and $\hat y$ is the predicted output. As a computation graph, this looks like:

![Linear Regression Computation Graph](https://imgur.com/IcBhTjS.png)

PyTorch dynamically build the computational graph, for example
![DynamicGraph.gif](https://raw.githubusercontent.com/pytorch/pytorch/master/docs/source/_static/img/dynamic_graph.gif)

### Computing gradients with PyTorch
PyTorch allows to automatically obtain the gradients of a tensor with respect to a defined function. When creating the tensor, we have to indicate that it requires the gradient computation using the flag `requires_grad`  

In [ ]:
x = torch.rand(3,requires_grad=True)
print(x)

Notice that now the Tensor shows the flag `requires_grad` as True. We can also activate such a flag in a Tensor already created as follows:

In [ ]:
x = torch.tensor([1.0,2.0,3.0])
x.requires_grad_(True)
print(x)

Let us define a function $y=x^2+5$. The function $y$ will not only carry the result of evaluating $x$, but also the gradient function $\frac{\partial y}{\partial x}$ called `grad_fn` in the new tensor $y$

In [ ]:
x = torch.tensor([2.0])
x.requires_grad_(True)  #indicate we will need the gradients with respecto to this variable
y = x**2 + 5
print(y)

To evaluate the partial derivative $\frac{\partial y}{\partial x}$, we use the `.backward()` function and the result of the gradient evaluation is stored in `x.grad` 

In [ ]:
y.backward()  #dy/dx
print('PyTorch gradient:', x.grad)

#L et us compare with the analitical gradient of y = x**2+5
with torch.no_grad():    #this is to only use the tensor value without its gradient information
    dy_dx = 2*x  #analitical gradient
print('Analitical gradient:',dy_dx)

If we evaluate a vector $\mathbf{w}=[w_1, \ldots, w_D]^{\top}$, to compute another vector $\mathbf{g}=[g_1, \ldots, g_D]^{\top}$ with elements $g_i=w_i^2+5$, then we obtain a vector $\mathbf{g}$ that contains each evaluation of the function. If we want to obtain the gradient w.r.t $\mathbf{w}$ by using "g.backward()", we have to bypass a vector of size equal to w.shape to the function, i.e., "g.backward(vect)". 

In [ ]:
w = torch.tensor([1.0,2.0,3.0])
w.requires_grad_(True)

g = w**2+5
# Below, the values [1.0,1.0,1.0] are multiplied by the gradient g.backward(vect)
# of course using the ones does not modify the value of the gradient
vect = torch.tensor([1.0,1.0,1.0],dtype=torch.float32) 
g.backward(vect)
print(w.grad)

On the other hand, when accessing the gradients in a for loop, PyTorch acummulates the gradients at each
iteration. In order to avoid this behaviour, we have to use the function .grad.zero_() also at each iteration. See in the example below what happens when commenting and uncommenting the line "w.grad.zero_()":

In [ ]:
#Pytorch uses a cumulative process for the gradients
w = torch.tensor([1.0,2.0,3.0])
w.requires_grad_(True)

for i in range(3):
    g = w**2+5
    g.backward(torch.ones_like(w))
    print(w.grad)
    w.grad.zero_()    #this line avoids the acummulation of the gradients uncomment it to see its effect

###  Question 5

Verify that the gradients provided by PyTorch coincide with the *analytical* gradients of the function $f(x) = \exp \big(-x^2-2x- \sin (x) \big)$ w.r.t $x$.

In [ ]:
# Provide your answer here

## B4: Linear Regression Basic Example

We now provide a very simple example of linear regression with one input dimension, $y=wx+b$, and illustrate how we use PyTorch to optimise the parameters of the model

In [ ]:
Ndata = 100 
x = torch.rand(Ndata)
true_w = 1.5
true_bias = 1.0
# We generate the dataset from the actual model but adding some noise
y = true_w*x + true_bias + 0.05*torch.randn(Ndata)
# We make sure to set the requires_grad flag to True for both paratemers
w = torch.tensor(0.0,dtype=torch.float32,requires_grad=True)
bias = torch.tensor(0.0,dtype=torch.float32,requires_grad=True)

We now define two useful functions, the prediction function and the objective function

In [ ]:
def model_prediction(x,w,bias):
    return w*x + bias

def loss_function(y,y_pred):
    return ((y_pred-y)**2).mean()  #Mean Squared Error (MSE)

And we use coordinate descent to estimate the parameters of the model

\begin{align*}
    w_{k+1} = w_k - \eta \frac{dE}{dw}\\ 
    b_{k+1} = b_k - \eta \frac{dE}{db}\\ 
\end{align*}

We know that there is a closed form solution for $w$ and $b$ through the normal equation. The example is for illustrative purposes.

In [ ]:
Max_Niter = 500
step_size = 0.1
for Niter in range(Max_Niter):
    # Evaluate the prediction and the loss
    y_approx = model_prediction(x,w,bias)
    my_loss = loss_function(y,y_approx)
    
    # The function .backward() has to be called in order to load the grads in w.grad
    # Notice that here it is not necessary to bypass a vector since loss_function is a scalar function
    my_loss.backward()  
        
    with torch.no_grad():        # this line avoids the gradient update while allowing to change the value of w
        w -= step_size*w.grad    # it is necessary to avoid the grad update while modifying the variable
        bias -= step_size*bias.grad
    
    # Make the zero gradient to avoid acummulation
    w.grad.zero_()
    bias.grad.zero_()
    
    # We print the loss, and the parameters values every 50 iterations
    if Niter%50==0:
        print(f'Iteration = {Niter+1}, Loss = {my_loss:.8f}, w = {w:.3f}, bias = {bias:.3f}')        

print(f'Iteration = {Niter+1}, Loss = {my_loss:.8f}, w = {w:.3f}, bias = {bias:.3f}')    
    

We finally plot the result

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(x,y,'x')
xtest = torch.linspace(0,1,10)
with torch.no_grad():
    y_pred = model_prediction(xtest,w,bias)
plt.plot(xtest,y_pred)

## B5: Revisiting linear regression for the Rented Bike Dataset of lab notebook 2

We will implement a linear regression for the Rented Bike dataset previously used in Lab. 2. We will use the same data preparation through `sklearn.preprocessing`: the OneHotEncoder() that allows to transform a categorical variable to a one-hot encoding representation, and StandardScaler() performs feature scaling by standardisation.

In [ ]:
import urllib.request
urllib.request.urlretrieve('https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv', './SeoulBikeData.csv')

The following code was borrowed from Lab Notebook 2. You can go back to that Notebook for details.

In [ ]:
import pandas as pd 

bike_sharing_data = pd.read_csv('SeoulBikeData.csv', encoding= 'unicode_escape')
bike_sharing_data = bike_sharing_data.drop('Date', axis=1)

for col in ['Rented Bike Count', 'Hour', 'Humidity(%)', 'Visibility (10m)']:
    bike_sharing_data[col] = bike_sharing_data[col].astype('float64')

attributes_cat = ['Seasons', 'Holiday', 'Functioning Day']
attributes_num = ['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)', \
                  'Dew point temperature(°C)', 'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)']

# We split our dataset for Training and Testing

from sklearn.model_selection import train_test_split
bs_train_set, bs_test_set = train_test_split(bike_sharing_data, test_size=0.15, random_state=42)
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

full_transform = ColumnTransformer([
    ("num", StandardScaler(), attributes_num),
    ("cat", OneHotEncoder(), attributes_cat),
])

# We separate the features from the labels

bs_train_set_attributes = bs_train_set.drop('Rented Bike Count', axis=1)
bs_test_set_attributes = bs_test_set.drop('Rented Bike Count', axis=1)
bs_train_set_labels = bs_train_set['Rented Bike Count']
bs_test_set_labels = bs_test_set['Rented Bike Count']

We now use the function `torch.from_numpy()` to transform the data previously prepared, into a Torch Tensor. We make sure to add a column of ones to the attributes (remember that $x_0=1$) both in the train and test sets.

In [ ]:
# We apply the preprocessing transformation over the features of the training data

bs_train_set_attributes_prepared = full_transform.fit_transform(bs_train_set_attributes)
bs_test_set_attributes_prepared = full_transform.transform(bs_test_set_attributes)

Train_torch = torch.from_numpy(bs_train_set_attributes_prepared)

# The line below adds a feature vector of ones in order to allow the bias weight
# to be represented in a unique weight vector.

Train_torch = torch.cat((torch.ones([Train_torch.shape[0],1],dtype=torch.float64),Train_torch), 1)  
Test_torch = torch.from_numpy(bs_test_set_attributes_prepared)

# The line below adds a feature vector of ones in order to allow the bias weight
# to be represented in a unique weight vector.

Test_torch = torch.cat((torch.ones([Test_torch.shape[0],1],dtype=torch.float64),Test_torch), 1)
Train_Label_torch = torch.from_numpy(bs_train_set_labels.values)

Test_Label_torch = torch.from_numpy(bs_test_set_labels.values)

We create a vector of weights $\mathbf{w}$ with the corresponding flag for the gradient and two functions, one for prediction and one for the loss function.

In [ ]:
# We create the vector of weights to be optimised in the linear regression model
dim = Train_torch.shape[1]
w = torch.randn([dim,1],dtype=torch.float64)  # vector of weight w is a vector Dim x 1
w.requires_grad_(True)

# We create the model prediction which consists on an inner product X'w, where X is a design matrix of N x Dim
def model_prediction_lr(x,w):
    return torch.matmul(x,w)

def loss_function_lr(y,y_pred):
    return ((y_pred-y)**2).mean()  # Mean Squared Error (MSE)

We finally use gradient descent to find the optimal value for $\mathbf{w}$
$$
\mathbf{w}_{k+1} = \mathbf{w}_k - \eta \frac{dE(\mathbf{w})}{d\mathbf{w}}
$$


In [ ]:
# Training the model with Gradient Descent

Max_Niter = 50 # If you have many iterations, this process can take some time
step_size = 0.001
for Niter in range(Max_Niter):
    y_approx = model_prediction_lr(Train_torch,w)
    my_loss = loss_function_lr(Train_Label_torch,y_approx)
    
    # The function .backward() has to be called in order to load the grads in w.grad
    # Notice that here it is not necessary to bypass a vector since loss_function is a scalar function
    
    my_loss.backward()  
        
    with torch.no_grad():    # this line avoids the gradient update while allowing to change the value of w
        w -= step_size*w.grad    # it is necessary to avoid the grad update while modifying the variable
        
    # print(w.grad)
    # Make the zero gradient to avoid acummulation
    w.grad.zero_()
    
    if Niter%20==0 or Niter == Max_Niter-1:
        print(f'Iteration = {Niter+1}, Loss = {my_loss:.8f}')
        #print('Weights vector:\n', w)

We finally provide the RMSE for the test set

In [ ]:
# RMSE over the test set

y_pred_test = model_prediction_lr(Test_torch,w)
MSE_test = loss_function_lr(Test_Label_torch,y_pred_test)
print('The Root Mean Squared Error over the test set is:', np.sqrt(MSE_test.detach().numpy()))

## B6: Using torch.nn and torch.optim

In the above sections we have performed a lot of the training steps by hand but PyTorch provides many useful implementations of these steps. In this section, we will use the torch.nn and torch.optim parts to simplify the training.

### torch.nn
Let's start with the nn part of the library. This implements many building blocks for neural networks and graphs. This includes linear layers and non-linear activation functions. We will start to use more of this next time but this will hopefully get you started with it. 


In [ ]:
from torch import nn


First let's have a look at a linear layer. This is simply the operation that we have been performing with $\mathbf{w} \mathbf{x}$ i.e it is a linear combination of the weight parameters and the inputs. The [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) class will set up and apply this operation for us. Internally it holds weights and bias parameters that we will learn in linear regression. Let's test it out with a simple 2 inputs and 1 output.

In [ ]:
# The weights and biases will be initialised randomly, so let's fix the seed to get the same values.
torch.manual_seed(12345)

# Create an instance of a Linear layer (setting the bias to be included)
lin = nn.Linear( in_features = 2, out_features = 1, bias=True)

print("The initial weights are:\n", lin.weight)
print("The initial bias is:\n", lin.bias)
# Note that when we print these it tells us that it is a Parameter. This a special type indicator that will be used for training.

#Let's test it out. It has an () operator that will apply the operation to the input.
input = torch.rand([2])
print("Input values are:\n", input)
print("The result of the linear layer is:\n", lin(input))

# Calculate the result for your self, is it correct?

### torch.nn.functional

nn.Linear allowed us to implement the linear summation but we can also use the [nn.functional](https://pytorch.org/docs/stable/nn.functional.html) library. Again there are many options here but let's just consider replacing our loss function with one implemented by PyTorch. Earlier we used the mean squared error so we can do the same here. Note that it is also implemented as a class in torch.nn but this gives us the direction function.




In [ ]:
import torch.nn.functional as F

loss_func = F.mse_loss

#Let's try it out
predictions = torch.rand([3])
targets = torch.rand([3])

print(predictions)
print(targets)
print("PyTorch loss function:", loss_func(predictions, targets))
print("Our earlier loss function:", loss_function_lr(predictions, targets))


## torch.optim

When we have many parameters (weights, biases, etc) it can be tricky keeping track of them individually. The [torch.optim](https://pytorch.org/docs/stable/optim.html) library helps with this by automatically applying our gradient descent weight updates. There is also a variety of update tools available, for example Adam is a widely used learning rule. We will use the stochastic gradient descent (SGD) as earlier. 

In [ ]:
from torch import optim

# When we initialise the SGD class we need to tell it what parameters we want to update and the learning rate.
# Here we will just use the linear layer from above and step_size from before.
opt = optim.SGD(lin.parameters(), lr = step_size)

# Since the parameters are linked to the optimiser we can use that to zero them all out
opt.zero_grad()

# So now if we use our linear layer as a model to accumulate the gradients
targets = torch.rand([1])
predictions = lin(input)
loss = loss_func(predictions, targets)
loss.backward()

print("Our gradients are:", lin.weight.grad)

# Now when we want to update the parameters we can just call
opt.step()

print('Our updated weights are:\n', lin.weight)
print('Our updated bias is:\n', lin.bias)

# And remember to zero the gradients to stop multiple steps building up (unless that's what you want, see batches)
opt.zero_grad()




## Putting them together
Let's now use this new functionality to simplify our training routine from earlier.

In [ ]:
# Training the model with Gradient Descent

Max_Niter = 50 # If you have many iterations, this process can take some time
step_size = 0.001

# Create a linear model with the correct number of input features
# Here we will turn the bias off since we didn't have it before
# By default, Linear will be float32 (single precision) dtype but our training
# data is float64 (double) so we will need to initialise it correctly.
model = nn.Linear(in_features = dim, out_features = 1, bias=False, dtype=Train_torch.dtype)

# Set up our optimiser, linked to the model parameters and step size.
opt = optim.SGD(model.parameters(), lr=step_size)

# nn.Linear is giving us an output which has a shape (number_of_samples, 1) but our labels are 1d.
# To stop errors lets reshape our label array.
Train_Label_torch = Train_Label_torch.reshape((-1,1))
print(Train_Label_torch.shape)

for Niter in range(Max_Niter):
    y_approx = model(Train_torch)
    my_loss = loss_func(y_approx, Train_Label_torch)
    
    my_loss.backward()  

    opt.step()
    opt.zero_grad()
        
    if Niter%20==0 or Niter == Max_Niter-1:
        print(f'Iteration = {Niter+1}, Loss = {my_loss:.8f}')
        #print('Weights vector:\n', w)

Try changing the number of max iterations (epochs) to see how low you can get the loss. While it is performing the same operations as before this will set us up for more complicated models.

---
## B6: Logistic regression with PyTorch

Now that we are more familiar let's try implementing logistic regression using PyTorch rather than Scikit-Learn. The implementation is very similar to linear regression but we need to modify our model and loss function. 

For converting the output of the linear layer to a probability you will need to use the [logistic sigmoid](https://pytorch.org/docs/stable/generated/torch.nn.functional.sigmoid.html#torch.nn.functional.sigmoid).
Instead of the mse_loss, the logistic regression uses the negative log likelihood. Another name for this is [binary cross entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.functional.binary_cross_entropy.html#torch.nn.functional.binary_cross_entropy). 



### Question 6

Implement logistic regression using PyTorch (the `torch.nn.`) and apply it to the synthetic 2D data in part A (or a real dataset if you want it to be more challenging) for classification. You may also vary the synthetic data to observe performance variation. Check out [**reproducibility** in PyTorch](https://pytorch.org/docs/stable/notes/randomness.html).

In [ ]:
# Provide your answer here

## Additional ideas to explore

* Change the [loss function](https://pytorch.org/docs/stable/nn.html#loss-functions) to different choices and compare the results.  
* Formulate another regression problem and solve it using `torch.nn`
* Compare the `torch.nn` solution against the closed-form solution
* Explore any other variations that you can think of to learn more
* Explore more advanced examples at the [PyKale library](https://github.com/pykale/pykale/tree/master/examples) 